### 이상치 
- 결측치, 범위에서 벗어나는 데이터, 값이 크게 벗어나는 데이터
- 분석 모델의 성능을 떨어뜨리거나 결과에 악영향을 미치는 경우가 발생
- 이상치를 발견하면 확인 후 특정한 값을 대체, 삭제
- IQR 방식
    - matplotlib 에서 boxplot의 이상치 결정 방법
    - Q3(3사분위수)와 Q1(1사분위수)를 이용해서 범위 지정 
    - IQR = Q3 - Q1
    - 이상치의 경계 (Q3 + 1.5*IQR) (Q1 - 1.5 * IQR)
    - 많은 데이터를 이상치 판단할 수 있다. 
    - 1.5 값을 조절해서 이상치의 범위를 변경

In [ ]:
# 사이킷런 설치 
# !pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine


In [ ]:
# 데이터셋 로드 
wine_load = load_wine()

In [ ]:
wine_load

In [ ]:
# wine_load에 data를 이용하여 데이터프레임을 생성 
wine = pd.DataFrame(wine_load['data'], columns=wine_load['feature_names'] )
wine.head() 

In [ ]:
wine['class'] = wine_load['target']

In [ ]:
wine.columns

In [ ]:
plt.boxplot(wine['color_intensity'])
plt.show()

In [ ]:
wine['color_intensity'].describe()

In [ ]:
q_1, q_3 = np.percentile(wine['color_intensity'], [25, 75])

In [ ]:
print(q_1)
print(q_3)

In [ ]:
iqr = q_3 - q_1
upper_whis = q_3 + (1.5 * iqr)
lower_whis = q_1 - (1.5 * iqr)
print(upper_whis, lower_whis)

- iqr을 이용한 이상치 판단 함수 생성 
    - 매개변수 3개
        - DataFrame형태의 데이터 
        - 기준이되는 컬럼(갯수가 가변)
        - cnt (범위 기본값을 1.5)
    - 기준이 되는 컬럼의 개수만큼 반복 실행 
        - 컬럼의 데이터를 기준으로 1사분위수, 3사분위수를 변수에 저장 
        - iqr 변수를 생성 
        - 상단의 경계, 하단의 경계를 지정 
        - 데이터프레임에서 해당하는 경계를 이용하여 이상치를 출력
        - 딕셔너리에 대입 (key값은 컬럼의 이름, value는 데이터프레임)
    - 딕셔너리 데이터를 되돌려준다. 

In [ ]:
def outliers_iqr(_df, *_cols, _cnt = 1.5):
    # 빈 딕셔너리 데이터를 생성 
    result = dict()
    # _cols은 데이터의 타입? -> tuple
    for col in _cols:
        # 1사분위, 3사분위 데이터를 생성 
        q_1, q_3 = np.percentile( _df[col], [25, 75] )
        # iqr 생성
        iqr = q_3 - q_1
        # 상단 경계 하단 경계 생성 
        upper_whis = q_3 + ( _cnt * iqr )
        lower_whis = q_1 - ( _cnt * iqr )
        print(f"""
            {col}컬럼의 데이터에 
            상단의 경계는{upper_whis}이고 
            하단의 경계는{lower_whis}이다
            """)
        # 상단 경계와 하단경계를 이용하여 이상치 데이터를 추출
        outlier = _df[ (_df[col] > upper_whis) | (_df[col] < lower_whis) ]
        # 빅 딕셔너리에 대입 
        result[col] = outlier
    return result

In [ ]:
wine.columns

In [24]:
outliers = outliers_iqr(wine, 'color_intensity', 'proanthocyanins')


            color_intensity컬럼의 데이터에 
            상단의 경계는10.670000000000002이고 
            하단의 경계는-1.2500000000000009이다
            

            proanthocyanins컬럼의 데이터에 
            상단의 경계는3.0이고 
            하단의 경계는0.20000000000000018이다
            


In [ ]:
outliers['color_intensity']

In [ ]:
outliers['proanthocyanins']

### 이상치 정제 
1. 이상치 제거 
    - 데이터의 개수가 매우 많고 이상치가 존재하는 데이터의 개수가 전체 데이터에 비해 많지 않은 경우에는 제거 
    - 이상치를 제거 -> 데이터의 개수가 줄어든다. 
2. 이상치를 특정 데이터로 대체
    - 이상치의 값을 다른 값을 대체하는경우는 데이터 분석가에 따라 값들의 차이

In [28]:
df = wine.copy()

In [30]:
# 이상치를 제거 
df.drop(outliers['color_intensity'].index, axis=0, inplace=True)

In [31]:
print(f"원본데이터의 길이 : {len(wine)}")
print(f"이상치가 제거된 데이터의 길이 : {len(df)}")

원본데이터의 길이 : 178
이상치가 제거된 데이터의 길이 : 174


In [34]:
df2 = wine.copy()

In [35]:
# 이상치 데이터를 특정 데이터로 변경 
df2.loc[outliers['color_intensity'].index, 'color_intensity'] = \
    df2['color_intensity'].mean()

In [36]:
df2.loc[outliers['color_intensity'].index, 'color_intensity']

151    5.05809
158    5.05809
159    5.05809
166    5.05809
Name: color_intensity, dtype: float64

In [37]:
upper_whis = 10.67
lower_whis = -1.25

In [44]:
wine.loc[outliers['color_intensity'].index, 'color_intensity']

151    10.67
158    10.67
159    10.67
166    10.67
Name: color_intensity, dtype: float64

In [43]:
for idx in outliers['color_intensity'].index:
    # print(idx)
    data = wine.loc[idx, 'color_intensity']
    # print(data)
    if data > upper_whis:
        # print(upper_whis)
        wine.loc[idx, 'color_intensity'] = upper_whis
    elif data < lower_whis:
        # print(lower_whis)
        wine.loc[idx, 'color_intensity'] = lower_whis

    # break

In [45]:
wine.loc[outliers['color_intensity'].index, 'color_intensity']

151    10.67
158    10.67
159    10.67
166    10.67
Name: color_intensity, dtype: float64

### 범주형 변수 처리
- 범주형 변수는 같이 수학적 연산으로 모델을 생성하는 대부분 분석 도구에서 직접적으로 사용하기 어렵기때문에 특별 가공 처리
- 더미변수 사용
- 더미변수는 범주형 변수에 있는 범주들을 각각 컬럼으로 변경 -> 0과 1로 데이터를 대입하여 포함 여부를 보여주는 변수

In [46]:
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [47]:
wine['class'] = wine['class'].map(
    {
        0 : 'Class_0', 
        1 : "Class_1", 
        2 : 'Class_2'
    }
)

In [49]:
wine['class'].value_counts()

class
Class_1    71
Class_0    59
Class_2    48
Name: count, dtype: int64

In [52]:
# 더미변수 생성 
# get_dummies() 함수 이용
wine_dummy = pd.get_dummies(wine, columns=['class'])

In [55]:
wine_dummy['class_Class_2'].sum()

48

### 데이터의 분할
- 분석 모델을 학습하고 성과를 확인하기 위하여 데이터를 train과 test 세트로 나누고 독립변수와 종속변수로 분리하는 작업 
- 분석 방법에 따라 train, test로 나누거나 train, validation, test로 분리하기도 함
    - train : test = 7 : 3
    - train : validation : test = 6 : 2 : 2
- sklearn 라이브러리에서 train_test_split() 함수를 이용하여 분할
    - train_test_split(X, Y, test_size = None, random_state = None, shuffle = True, stratify = None)
        - X : 독립 변수 데이터프레임
        - Y : 종속 변수 데이터프레임
        - test_size : test 데이터의 비율 (0 ~ 1) 
        - random_state : 임의의 번호를 지정, 같은 숫자를 사용한다면 같은 출력이 생성
        - shuffle : True라면 데이터를 추출하기 전에 섞어준다. (순서를 변경)
        - straify : None이 아닌 경우 데이터는 지정한 변수를 기준으로 계층화되어 해당 변수의 비율을 유지하도록 추출 

In [56]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [57]:
iris_load = load_iris()

In [61]:
iris_load.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [ ]:
iris = pd.DataFrame(iris_load['data'], columns=iris_load['feature_names'])
iris['class'] = iris_load['target']
iris.head()

In [63]:
iris['class'] = iris['class'].map({
    0 : 'Setosa', 
    1 : 'Versicolor', 
    2 : 'Virginica'
})

In [64]:
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [65]:
iris['class'].value_counts()

class
Setosa        50
Versicolor    50
Virginica     50
Name: count, dtype: int64

In [68]:
x_train, x_test, y_train, y_test = train_test_split(
    iris.drop('class', axis=1), 
    iris['class'], 
    test_size= 0.2, 
    random_state= 1000
)

In [69]:
print(f"학습데이터의 수 : {len(x_train)} 테스트데이터의 수 : {len(x_test)}")
print(f"종속변수 학습데이터의 수 : {len(y_train)}, 테스트 데이터의 수 : {len(y_test)}")

학습데이터의 수 : 120 테스트데이터의 수 : 30
종속변수 학습데이터의 수 : 120, 테스트 데이터의 수 : 30


In [72]:
y_train.value_counts()

class
Setosa        41
Virginica     41
Versicolor    38
Name: count, dtype: int64

In [86]:
X_train, X_test, Y_train, Y_test = train_test_split(
    iris.drop('class', axis=1), 
    iris['class'], 
    test_size= 0.2, 
    stratify=iris['class'], 
    shuffle= True
)

c:\Users\moons\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\moons\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [87]:
Y_train.value_counts()

class
Setosa        40
Virginica     40
Versicolor    40
Name: count, dtype: int64

In [88]:
X_test

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
137,6.4,3.1,5.5,1.8
16,5.4,3.9,1.3,0.4
126,6.2,2.8,4.8,1.8
78,6.0,2.9,4.5,1.5
136,6.3,3.4,5.6,2.4
84,5.4,3.0,4.5,1.5
61,5.9,3.0,4.2,1.5
18,5.7,3.8,1.7,0.3
90,5.5,2.6,4.4,1.2
65,6.7,3.1,4.4,1.4
